In [1]:
from pyspark.sql import SparkSession, functions, types
spark = SparkSession.builder.appName('temp').getOrCreate()
assert spark.version >= '2.4' # make sure we have Spark 2.4+
spark.sparkContext.setLogLevel('WARN')
sc = spark.sparkContext

In [19]:
import pandas as pd
from dataprep.eda import plot

NumExpr defaulting to 4 threads.


In [3]:
patient = pd.read_csv("mimic/PATIENTS.csv")

In [5]:
admission = pd.read_csv("mimic/ADMISSIONS.csv")
diagnoses_d = pd.read_csv("mimic/D_ICD_DIAGNOSES.csv")
diagnoses = pd.read_csv("mimic/DIAGNOSES_ICD.csv")
icu = pd.read_csv("mimic/ICUSTAYS.csv")
prescriptions = pd.read_csv("mimic/PRESCRIPTIONS.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
admission = admission.drop(['ADMITTIME', 'DISCHTIME', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'EDREGTIME', 'EDOUTTIME', 'HAS_CHARTEVENTS_DATA'], axis=1)
admission

,ROW_ID,SUBJECT_ID,HADM_ID,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG
0,21,22,165315,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,WHITE,BENZODIAZEPINE OVERDOSE,0
1,22,23,152223,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,WHITE,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0
2,23,23,124321,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,WHITE,BRAIN MASS,0
3,24,24,161859,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,WHITE,INTERIOR MYOCARDIAL INFARCTION,0
4,25,25,129635,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,WHITE,ACUTE CORONARY SYNDROME,0
...,...,...,...,...,...,...,...,...,...
58971,58594,98800,191113,NaN,EMERGENCY,CLINIC REFERRAL/PREMATURE,WHITE,TRAUMA,0
58972,58595,98802,101071,2151-03-06 09:10:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,WHITE,SAH,1
58973,58596,98805,122631,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,WHITE,RENAL CANCER/SDA,0
58974,58597,98813,170407,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,WHITE,S/P FALL,0


,ROW_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
SUBJECT_ID,,,,,,,,,,,,,,,,,,
2,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1
3,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1
4,1,1,1,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1
5,1,1,1,1,0,1,1,1,1,0,1,0,1,0,0,1,1,1
6,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99985,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1
99991,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,1,1,1
99992,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1


In [30]:
# diagnosis - mortality
dead = admission[admission['HOSPITAL_EXPIRE_FLAG']==1]
dead

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,STATUS EPILEPTICUS,1,1
57,55,56,181711,2104-01-02 02:01:00,2104-01-08 10:30:00,2104-01-08 10:30:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,NOT SPECIFIED,NaN,WHITE,2104-01-01 23:59:00,2104-01-02 03:33:00,HEAD BLEED,1,1
63,61,61,189535,2119-01-04 18:12:00,2119-02-03 01:35:00,2119-02-03 01:35:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Private,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,1,1
70,68,67,155252,2157-12-02 00:45:00,2157-12-02 03:55:00,2157-12-02 03:55:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,NaN,JEWISH,SINGLE,WHITE,2157-12-01 20:45:00,2157-12-02 00:55:00,SUBARACHNOID HEMORRHAGE,1,1
88,86,84,166401,2196-04-14 04:02:00,2196-04-17 13:42:00,2196-04-17 13:42:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Private,NaN,OTHER,MARRIED,WHITE,2196-04-13 22:23:00,2196-04-14 04:31:00,"GLIOBLASTOMA,NAUSEA",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58927,58550,98669,108710,2147-04-07 15:09:00,2147-04-14 10:00:00,2147-04-14 10:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,UNOBTAINABLE,NaN,UNABLE TO OBTAIN,2147-04-07 13:31:00,2147-04-07 16:55:00,SUBARACHNOID HEMORRHAGE,1,1
58957,58580,98753,185764,2172-02-11 21:26:00,2172-03-02 08:00:00,2172-03-02 08:00:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,2172-02-11 16:49:00,2172-02-11 22:52:00,R/O METASTASIS,1,1
58966,58589,98768,127022,2112-09-15 09:45:00,2112-09-17 08:20:00,2112-09-17 08:20:00,EMERGENCY,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2112-09-15 09:30:00,2112-09-15 11:46:00,LEUKOCYTOSIS,1,1
58970,58593,98797,105447,2132-12-24 20:06:00,2132-12-25 12:00:00,2132-12-25 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2132-12-24 18:53:00,2132-12-24 21:41:00,ALTERED MENTAL STATUS,1,1


In [20]:
plot(admission)

GridBox(id='1728', ...)

In [29]:
plot(diagnoses)

GridBox(id='2723', ...)

In [47]:
category = {
    0: 'infectious and parasitic diseases',
    1: 'neoplasms',
    2: 'endocrine, nutritional and metabolic diseases, and immunity disorders',
    3: 'diseases of the blood and blood-forming organs',
    4: 'mental disorders',
    5: 'diseases of the nervous system and sense organs',
    6: 'diseases of the circulatory system',
    7: 'diseases of the respiratory system',
    8: 'diseases of the digestive system',
    9: 'diseases of the genitourinary system',
    10: 'complications of pregnancy, childbirth, and the puerperium',
    11: 'diseases of the skin and subcutaneous tissue',
    12: 'diseases of the musculoskeletal system and connective tissue',
    13: 'congenital anomalies',
    14: 'certain conditions originating in the perinatal period',
    15: 'symptoms, signs, and ill-defined conditions',
    16: ' injury and poisoning',
    17: 'external causes of injury and supplemental classification'
}
def assign_category(icd9_code):
    try:
        code = int(icd9_code)
        if code >= 10000:
            code = code // 100
        elif code >= 1000:
            code = code // 10
        else:
            pass
        if code <= 139:
            return 0
        if code <= 239:
            return 1
        if code <= 279:
            return 2
        if code <= 289:
            return 3
        if code <= 319:
            return 4
        if code <= 389:
            return 5
        if code <= 459:
            return 6
        if code <= 519:
            return 7
        if code <= 579:
            return 8
        if code <= 629:
            return 9
        if code <= 679:
            return 10
        if code <= 709:
            return 11
        if code <= 739:
            return 12
        if code <= 759:
            return 13
        if code <= 779:
            return 14
        if code <= 799:
            return 15
        return 16
    except:
        return 17
diagnoses['category'] = diagnoses['ICD9_CODE'].apply(assign_category)
plot(diagnoses)

GridBox(id='3283', ...)

In [41]:
diagnoses

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [18]:
icu

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202
...,...,...,...,...,...,...,...,...,...,...,...,...
61527,59806,94944,143774,201233,metavision,CSRU,CSRU,15,15,2104-04-15 10:18:16,2104-04-17 14:51:00,2.1894
61528,59807,94950,123750,283653,metavision,CCU,CCU,7,7,2155-12-08 05:33:16,2155-12-10 17:24:58,2.4942
61529,59808,94953,196881,241585,metavision,SICU,SICU,57,57,2160-03-03 16:09:11,2160-03-04 14:22:33,0.9259
61530,59809,94954,118475,202802,metavision,CSRU,CSRU,15,15,2183-03-25 09:53:10,2183-03-27 17:55:03,2.3346


In [22]:
drg = pd.read_csv("mimic/DRGCODES.csv")

In [27]:
t = drg.groupby('DRG_CODE').size()

In [28]:
t

DRG_CODE
1       626
2       150
3       461
4       345
5       174
       ... 
9521      7
9522     19
9523     39
9524     62
9550      1
Length: 1667, dtype: int64